In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Dataset_Inspection_and_Cleaning").getOrCreate()

# Load the dataset with updated options
file_path = "/content/Sampled_Dataset.csv"
data = spark.read.csv(
    file_path,
    header=True,
    inferSchema=True,
    multiLine=True,  # Handles multiline data in cells
    escape='"',      # Handles escaped quotes within data
    quote='"',       # Handles quoted strings properly
    encoding="UTF-8" # Ensures correct text encoding
)

# Show the schema to confirm proper loading
data.printSchema()

# Display the first few rows
data.show(truncate=False, n=20)

root
 |-- JD_Experience: string (nullable = true)
 |-- JD_Qualifications: string (nullable = true)
 |-- JD_Preference: string (nullable = true)
 |-- JD_Job Title: string (nullable = true)
 |-- JD_Role: string (nullable = true)
 |-- JD_Job Description: string (nullable = true)
 |-- JD_skills: string (nullable = true)
 |-- JD_Responsibilities: string (nullable = true)
 |-- Resume_Category: string (nullable = true)
 |-- Resume_information: string (nullable = true)

+-------------+-----------------+-------------+---------------------+-------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [2]:
# Number of rows
num_rows = data.count()

# Number of columns
num_cols = len(data.columns)

# Print the shape
print(f"Shape of the DataFrame: ({num_rows}, {num_cols})")

Shape of the DataFrame: (5690, 10)


In [3]:
from pyspark.sql.functions import col, regexp_extract, when
from pyspark.sql.types import IntegerType

# Step 1: Extract the minimum experience from the JD_Experience column
# Regex explanation: \d+ captures one or more digits (the numbers in the string)
data = data.withColumn("JD_Minimum_Experience",
                       regexp_extract(col("JD_Experience"), r"(\d+)", 1).cast(IntegerType()))

# Step 2: Replace nulls or missing values with 0 (if necessary)
data = data.withColumn("JD_Minimum_Experience",
                       when(col("JD_Minimum_Experience").isNull(), 0).otherwise(col("JD_Minimum_Experience")))

# Step 3: Drop the original JD_Experience column if it's no longer needed
data = data.drop("JD_Experience")

# Step 4: Display the updated DataFrame
data.select("JD_Minimum_Experience").show(5)

+---------------------+
|JD_Minimum_Experience|
+---------------------+
|                    4|
|                    1|
|                    0|
|                    2|
|                    5|
+---------------------+
only showing top 5 rows



In [4]:
from pyspark.sql.functions import col, sum

# Count the number of nulls in each column
null_counts = data.select(
    *[sum(col(column).isNull().cast("int")).alias(column) for column in data.columns]
)

# Show the null count for each column
null_counts.show()

+-----------------+-------------+------------+-------+------------------+---------+-------------------+---------------+------------------+---------------------+
|JD_Qualifications|JD_Preference|JD_Job Title|JD_Role|JD_Job Description|JD_skills|JD_Responsibilities|Resume_Category|Resume_information|JD_Minimum_Experience|
+-----------------+-------------+------------+-------+------------------+---------+-------------------+---------------+------------------+---------------------+
|                0|            0|           0|      0|                 0|        0|                  0|              0|                 0|                    0|
+-----------------+-------------+------------+-------+------------------+---------+-------------------+---------------+------------------+---------------------+



In [5]:
from pyspark.sql.functions import col, sum, when

# Count empty strings for each column
empty_string_counts = data.select(
    *[sum(when(col(column) == "", 1).otherwise(0)).alias(column) for column in data.columns]
)

# Show counts of empty strings
empty_string_counts.show()

+-----------------+-------------+------------+-------+------------------+---------+-------------------+---------------+------------------+---------------------+
|JD_Qualifications|JD_Preference|JD_Job Title|JD_Role|JD_Job Description|JD_skills|JD_Responsibilities|Resume_Category|Resume_information|JD_Minimum_Experience|
+-----------------+-------------+------------+-------+------------------+---------+-------------------+---------------+------------------+---------------------+
|                0|            0|           0|      0|                 0|        0|                  0|              0|                 0|                    0|
+-----------------+-------------+------------+-------+------------------+---------+-------------------+---------------+------------------+---------------------+



In [6]:
from pyspark.sql.functions import col, regexp_replace, trim, lower, when

# Columns to clean
columns_to_clean = [
    "JD_Qualifications", "JD_Preference", "JD_Job Title", "JD_Role",
    "JD_Job Description", "JD_skills", "JD_Responsibilities",
    "Resume_Category", "Resume_information"
]

# Step 1: Replace null values with an empty string
for column in columns_to_clean:
    data = data.withColumn(column, when(col(column).isNull(), "").otherwise(col(column)))

# Step 2: Clean the text
for column in columns_to_clean:
    data = data.withColumn(
        column,
        # Remove URLs, extra whitespace, and convert to lowercase
        lower(trim(regexp_replace(
            regexp_replace(col(column), r"https?://\S+|www\.\S+", ""),  # Remove URLs
            r"\s+", " "  # Replace multiple spaces with a single space
        )))
    )

# Show the cleaned DataFrame
data.select(columns_to_clean).show(truncate=False)

+-----------------+-------------+---------------------+-------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
from pyspark.sql.functions import regexp_replace, col

# List of columns to apply the correction
columns_to_correct = data.columns

# Correct the spelling "exprience" to "experience" across all columns
for column in columns_to_correct:
    data = data.withColumn(column, regexp_replace(col(column), r'\bexprience\b', 'experience'))

# Show the updated DataFrame
data.show(truncate=False)

+-----------------+-------------+---------------------+-------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
# Extract unique qualifications from the JD_Qualifications column
if "JD_Qualifications" in data.columns:
    unique_qualifications = data.select("JD_Qualifications").distinct().rdd.map(lambda row: row["JD_Qualifications"]).collect()
else:
    unique_qualifications = []

# Print the unique qualifications
print("Unique Qualifications in JD_Qualifications column:")
print(unique_qualifications)

Unique Qualifications in JD_Qualifications column:
['bca', 'phd', 'mca', 'ba', 'b.tech', 'm.tech', 'b.com', 'bba', 'm.com', 'mba']


In [10]:
import re
import spacy
from pyspark.sql.functions import udf, col, regexp_replace
from pyspark.sql.types import StringType

# Initialize SpaCy model
nlp = spacy.load("en_core_web_sm")

# Define the set of technical terms or keywords to preserve as-is
preserve_keywords = {"angular.js", "aws", "azure", "c#", "c++", "css", "django", "docker", "flask", "html",
    "java", "javascript", "kubernetes", "ms excel", "ms office", "ms power point",
    "node.js", "oracle", "pytorch", "python", "r", "react.js", "ruby", "sql",
    "tensorflow", "ui", "ux", "ux/ui", "bca", "phd", "mca", "ba", "mcom", "bcom", "bba", "mba", "btech", "mtech"}

# Define entity types to exclude
exclude_entities = {'ORG', 'GPE', 'LOC', 'DATE', 'TIME', 'PERSON', 'FAC', 'NORP', 'EVENT'}

# Function to clean and preserve keywords/entities
def preserve_and_clean_entities(text):
    # Handle null values
    if text is None:
        return ""

    # Remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Replace non-ASCII characters with a space
    doc = nlp(text)
    preserved_text = []

    for token in doc:
        # Check if the token is in the preserve_keywords set (case-insensitive check)
        if token.text.lower() in preserve_keywords:
            preserved_text.append(token.text)  # Preserve exactly as-is
        # Exclude entities based on entity types (ORG, GPE, etc.)
        elif token.ent_type_ in exclude_entities:
            continue  # Skip this token if it's an unwanted entity
        # Preserve technical entities identified as programming languages, technologies, etc.
        elif token.ent_type_ in ['PRODUCT', 'LANGUAGE']:
            preserved_text.append(token.text)
        # For other tokens, remove unnecessary punctuation and convert to lowercase
        elif token.is_alpha:
            preserved_text.append(token.text.lower())
        else:
            # Remove punctuation except those in preserve_keywords
            clean_token = re.sub(r'[^\w\s+#]', '', token.text)  # Retains + and # in technical terms
            if clean_token:  # Only add if token is not empty after cleaning
                preserved_text.append(clean_token.lower())

    # Join words and remove extra whitespace
    return " ".join(preserved_text).strip()

# Register the function as a UDF
clean_text_udf = udf(preserve_and_clean_entities, StringType())

# Step 1: Remove HTML tags and punctuations from all columns using regexp_replace
columns_to_clean = data.columns  # Apply to all columns
for column in columns_to_clean:
    data = data.withColumn(
        column,
        regexp_replace(  # Remove HTML tags
            regexp_replace(col(column), r"<[^>]+>", ""),  # Remove punctuations
            r"[^\w\s]", ""  # Retain only alphanumeric characters and whitespace
        )
    )

# Step 2: Apply the SpaCy-based cleaning to further clean and preserve technical terms
# Exclude 'JD_Experience' from additional cleaning if required
columns_to_clean = [col_name for col_name in data.columns if col_name != 'JD_Experience']

for column in columns_to_clean:
    data = data.withColumn(f"Cleaned_{column}", clean_text_udf(col(column)))

# Show the cleaned columns
data.select([col(f"Cleaned_{column}") for column in columns_to_clean]).show(truncate=False)

+-------------------------+---------------------+---------------------+-------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------